# Telemetry - Battery

In this notebook we will learn how to read telemetry data from a Duckiebot's battery using HTTP and then we will make a Battery component.

In [ ]:
%load_ext autoreload
%autoreload 2

# TODO: change this to the name of your Duckiebot
VEHICLE_NAME: str = "db21j3"

In [ ]:
# Telemetry - Battery - Read raw data from HTTP

import requests

# define API url
url: str = f"http://{VEHICLE_NAME}.local/health/battery"

# call API
response = requests.get(url)
data: dict = response.json()

# print response out
print(data)

In [ ]:
# Telemetry - Battery - Present results using the BatteryTelemetry class (defined in packages/duckietown/telemetry.py)

import requests
from duckietown.telemetry import BatteryTelemetry

# define API url
url: str = f"http://{VEHICLE_NAME}.local/health/battery"

# call API
response = requests.get(url)
data: dict = response.json()

# instantiate BatteryTelemetry
telemetry: BatteryTelemetry = BatteryTelemetry(**data["battery"])

# print response out
print(telemetry)

In [ ]:
# Telemetry - Battery - Wrap it inside a Component

import time
import requests
from duckietown.types import IQueue, Queue
from duckietown.components.base import Component
from duckietown.components.rendering import MarkdownRendererComponent
from duckietown.system import System


class BatteryTelemetryComponent(Component[None, BatteryTelemetry]):

    def __init__(self, vehicle_name: str, frequency: float):
        super(BatteryTelemetryComponent, self).__init__()
        assert frequency > 0
        # define url
        self._url: str = f"http://{vehicle_name}.local/health/battery"
        self._frequency: float = frequency
        # queues
        self.out_data: IQueue = Queue()
    
    def worker(self):
        while not self.is_shutdown:
            # call API
            response = requests.get(self._url)
            data: dict = response.json()
            # instantiate BatteryTelemetry
            telemetry: BatteryTelemetry = BatteryTelemetry(**data["battery"])
            # send data out
            self.out_data.put(telemetry)
            # wait
            time.sleep(1.0 / self._frequency)


def formatter(data: BatteryTelemetry) -> str:
    return f"""```json\n{data.as_json()}\n```"""


# test it by connecting it to a renderer node
battery: BatteryTelemetryComponent = BatteryTelemetryComponent(VEHICLE_NAME, 2)
renderer: MarkdownRendererComponent = MarkdownRendererComponent(formatter)

# connect components
renderer.in_data.wants(battery.out_data)

# create system
system: System = System([battery, renderer])

# run system (NOTE: this is blocking)
system.run()